## TODO
 * [ ] IIDX28まで更新する

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import collections
import json
from bs4 import BeautifulSoup

In [2]:
# # google colab only
# # After mounting google drive
# %cd drive/MyDrive/Colab\ Notebooks/dbdiff
# %ls modules
# !pip install js2py

/content/drive/MyDrive/Colab Notebooks/dbdiff
empty.html  generator.py  __init__.py  notes.py  __pycache__/  utils.py


In [2]:
import re
from modules import notes, utils
import js2py
import pdb

In [110]:
def get_scores_list(titletbl, actbl, playmode="dp"):
    ret_list = []
    for fbasename in list(titletbl):
        diffs = {}
        acin = {}
        ver, numNotes, opt, genre, arist, title, *_ = titletbl[fbasename]
        try:
            actbl_list = list(actbl[fbasename])
            print(len(actbl_list))
            subtitle = "" if len(actbl_list) <= 23 else actbl_list[23] 
            if playmode == "dp":
                acin["n"], acin["h"], acin["a"], acin["l"] = actbl_list[16:23:2]
                diffs["n"], diffs["h"], diffs["a"], diffs["l"] = actbl_list[15:22:2]
            elif playmode == "sp":
                acin["n"], acin["h"], acin["a"], acin["l"] = actbl_list[6:13:2]
                diffs["n"], diffs["h"], diffs["a"], diffs["l"] = actbl_list[5:12:2]
        except:
            continue
        for_add = [[fbasename, diffs[x], x, ver, bool(acin[x]&4), subtitle] for x in "nhal" if diffs[x] > 0]
        ret_list.extend(for_add)
    return ret_list

titletbl_path = "source/SCORE_OFF/titletbl.js"
actbl_path = "source/SCORE_OFF/actbl.js"

titletbl, actbl = utils.get_tables(titletbl_path, actbl_path)
scores = get_scores_list(titletbl, actbl, playmode="dp")

23
23
23
23
23
24
23
23
23
23
23
24
23
23
23
23
24
23
23
23
23
23
23
23
23
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
24
23
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
24
23
23
24
24
23
23
23
23
23
23
23
23
23
23
23
23
24
23
24
23
23
23
23
23
23
23
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
24
23
23
23
2

In [8]:
adjoin_key_idx = 3 # 1~7
mucha_sec = 60 / 190 / 4 # sec: BPM190の16分
keyNum = 7

def is_adjoin(adjoin_sp_list, playside):
    isAdjoin, isMuri = False, False
    for on_sc_list in adjoin_sp_list:
        if (playside == 1 and not all(x <= (adjoin_key_idx-1) for x in on_sc_list[0])) or \
           (playside == 2 and not all(x >= (keyNum-adjoin_key_idx-1) for x in on_sc_list[0])):
            isMuri = True
        elif (playside == 1 and not all(x > (adjoin_key_idx-1) for x in on_sc_list[0])) or \
             (playside == 2 and not all(x < (keyNum-adjoin_key_idx-1) for x in on_sc_list[0])):
            isAdjoin = True
        if isMuri and isAdjoin:
            break
    return isAdjoin, isMuri

def is_mucha(arr, sc_arr_sp, arr_bpm, playside):
    st, en = (1, 8) if playside == 1 else (8, -1)
    for idx in np.array(np.where(sc_arr_sp==1))[0]:
        bpm = arr_bpm[idx]
        range_dot = int(int(bpm) * mucha_sec * 4 / 5)
        if (2 <= arr[st:en, idx-range_dot:idx+range_dot+1, :].sum(axis=(0, 2)).astype(bool).astype(int).sum()):
            return True
    return False


def check_adjoin(arr, arr_bpm):
    sc_arr_1p = (arr[0,:,:] == 1).sum(axis=-1).astype(bool)
    sc_arr_2p = (arr[-1,:,:] == 1).sum(axis=-1).astype(bool)
    isMucha = is_mucha(arr, sc_arr_1p, arr_bpm, 1) or is_mucha(arr, sc_arr_2p, arr_bpm, 2)
    sc_on_1p = arr[1:8, sc_arr_1p, :].sum(axis=2) # 7 x scの枚数
    sc_on_2p = arr[8:-1, sc_arr_2p, :].sum(axis=2)
    hasAdjoin_1p, hasAdjoin_2p = sc_on_1p.sum() > 0, sc_on_2p.sum() > 0
    if not (hasAdjoin_1p or hasAdjoin_2p):
        if isMucha:
            return True, {"1p":{}, "2p":{}}, "無茶皿"
        else:
            return False, {}, None
    adjoin_1p_list = [np.array(np.where(x==1)).tolist() for x in sc_on_1p.transpose()]
    adjoin_2p_list = [np.array(np.where(x==1)).tolist() for x in sc_on_2p.transpose()]
    isAdjoin, isMuri = [x or y for x, y in zip(is_adjoin(adjoin_1p_list, 1), is_adjoin(adjoin_2p_list, 2))]
    judge_list = ["無茶皿", "隣接皿", "無理皿"]
    judge_bool_list = [isMucha, isAdjoin, isMuri]
    judge = ", ".join([judge_list[idx] for idx, x in enumerate(judge_bool_list) if x])
    adjoin_1p = collections.Counter(list(map(lambda x:",".join([str(y+1) for y in x[0]]), adjoin_1p_list)))
    adjoin_2p = collections.Counter(list(map(lambda x:",".join([str(y+1) for y in x[0]]), adjoin_2p_list)))
    del adjoin_1p[""]
    del adjoin_2p[""]
    return True, {"1p":dict(adjoin_1p), "2p":dict(adjoin_2p)}, judge


In [24]:
pp = "source/SCORE_OFF/20/_casino.html"
nt = notes.Notes(pp)
nt.convertNotes()
hasAdjoin, adjoins, judge = check_adjoin(nt.notesArray, nt.bpmArray)

In [25]:
judge

'無茶皿, 無理皿'

In [113]:
adjoin_scores = []
faild_name = []
for score in scores[:]:
    fbasename, lv, diff_char, ver, acin, subtitle = score
    parDir = os.path.join("source", "SCORE_OFF")
    ver = "s" if ver == "SS" else str(ver)
    fname = os.path.join(parDir, ver, fbasename+".html")
    try:
        nt = notes.Notes(fname, difficulty=diff_char)
        print("title: {0}, diff: {1}...".format(nt.title, nt.difficulty), end="")
        nt.convertNotes()
        hasAdjoin, adjoins, judge = check_adjoin(nt.notesArray, nt.bpmArray)
        if hasAdjoin:
            print("has adjoin !!!")
            adjoin_scores.append({"title":nt.title, "level":lv, "diff":diff_char, "adjoin_keys":adjoins, "judge":judge, "acin":acin, "subtitle":subtitle})
        elif not judge:
            print("no adjoin")
    except:
        faild_name.append(fname)
        continue


title: 100% minimoo-G, diff: n...no adjoin
title: 100% minimoo-G, diff: h...no adjoin
title: 100% minimoo-G, diff: a...has adjoin !!!
title: 1989, diff: n...no adjoin
title: 1989, diff: h...no adjoin
title: 19,November, diff: n...has adjoin !!!
title: 19,November, diff: h...has adjoin !!!
title: 19,November, diff: a...has adjoin !!!
title: 1st Samurai, diff: n...no adjoin
title: 1st Samurai, diff: h...no adjoin
title: 1st Samurai, diff: a...no adjoin
title: 2002, diff: n...no adjoin
title: 2002, diff: h...no adjoin
title: 20,November, diff: h...has adjoin !!!
title: 20,November, diff: a...has adjoin !!!
title: 20,November, diff: n...no adjoin
title: 20,November, diff: h...no adjoin
title: 20,November, diff: a...no adjoin
title: 22DUNK, diff: n...has adjoin !!!
title: 22DUNK, diff: h...has adjoin !!!
title: 22DUNK, diff: a...has adjoin !!!
title: 24th Century BOY, diff: n...no adjoin
title: 24th Century BOY, diff: h...no adjoin
title: 24th Century BOY, diff: a...no adjoin
title: 250bpm,

title: 華爛漫 -Flowers-, diff: a...no adjoin
title: 表裏一体!?怪盗いいんちょの悩み&hearts;, diff: n...no adjoin
title: 表裏一体!?怪盗いいんちょの悩み&hearts;, diff: h...has adjoin !!!
title: 表裏一体!?怪盗いいんちょの悩み&hearts;, diff: a...no adjoin
title: 一途な恋(HYPER J-EURO MIX), diff: n...no adjoin
title: 一途な恋(HYPER J-EURO MIX), diff: h...no adjoin
title: 一途な恋(HYPER J-EURO MIX), diff: a...no adjoin
title: 十六夜セツナ, diff: n...no adjoin
title: 十六夜セツナ, diff: h...no adjoin
title: 十六夜セツナ, diff: a...no adjoin
title: 炸裂！イェーガー電光チョップ!!, diff: n...has adjoin !!!
title: 炸裂！イェーガー電光チョップ!!, diff: h...has adjoin !!!
title: 炸裂！イェーガー電光チョップ!!, diff: a...has adjoin !!!
title: 時空トラベローグ, diff: n...no adjoin
title: 時空トラベローグ, diff: h...no adjoin
title: 時空トラベローグ, diff: a...no adjoin
title: 華蝶風雪, diff: n...no adjoin
title: 華蝶風雪, diff: h...has adjoin !!!
title: 華蝶風雪, diff: a...no adjoin
title: 蛇神, diff: n...no adjoin
title: 蛇神, diff: h...no adjoin
title: 蛇神, diff: a...no adjoin
title: 下弦の月, diff: n...no adjoin
title: 下弦の月, diff: h...no adjoin
title: 下弦の月,

title: マチ子の唄, diff: n...no adjoin
title: マチ子の唄, diff: h...no adjoin
title: マチ子の唄, diff: a...no adjoin
title: 桜, diff: n...no adjoin
title: 桜, diff: h...has adjoin !!!
title: 桜, diff: a...has adjoin !!!
title: シュレーディンガーの猫, diff: n...no adjoin
title: シュレーディンガーの猫, diff: h...no adjoin
title: シュレーディンガーの猫, diff: a...no adjoin
title: 聖人の塔, diff: n...no adjoin
title: 聖人の塔, diff: h...no adjoin
title: 聖人の塔, diff: a...no adjoin
title: 晴天Bon Voyage, diff: n...no adjoin
title: 晴天Bon Voyage, diff: h...no adjoin
title: 晴天Bon Voyage, diff: a...no adjoin
title: 千年ノ理, diff: n...no adjoin
title: 千年ノ理, diff: h...no adjoin
title: 千年ノ理, diff: a...has adjoin !!!
title: 灼熱Beach Side Bunny, diff: n...no adjoin
title: 灼熱Beach Side Bunny, diff: h...no adjoin
title: 灼熱Beach Side Bunny, diff: a...no adjoin
title: それは花火のような恋, diff: n...no adjoin
title: それは花火のような恋, diff: h...no adjoin
title: それは花火のような恋, diff: a...no adjoin
title: サヨナラ・ヘヴン, diff: n...no adjoin
title: サヨナラ・ヘヴン, diff: h...no adjoin
title: サヨナラ・ヘヴン, dif

title: Aegis, diff: h...has adjoin !!!
title: Aegis, diff: a...has adjoin !!!
title: AFRO KNUCKLE, diff: n...no adjoin
title: AFRO KNUCKLE, diff: h...no adjoin
title: AFRO KNUCKLE, diff: a...no adjoin
title: AGEHA, diff: n...no adjoin
title: AGEHA, diff: h...no adjoin
title: AGEHA, diff: a...no adjoin
title: Agnus Dei, diff: n...no adjoin
title: Agnus Dei, diff: h...no adjoin
title: Agnus Dei, diff: a...has adjoin !!!
title: Air Bell, diff: n...no adjoin
title: Air Bell, diff: h...no adjoin
title: Air Bell, diff: a...no adjoin
title: airflow, diff: n...no adjoin
title: airflow, diff: h...no adjoin
title: airflow, diff: a...no adjoin
title: AIR RAID FROM THA UNDAGROUND, diff: n...no adjoin
title: AIR RAID FROM THA UNDAGROUND, diff: h...no adjoin
title: AIR RAID FROM THA UNDAGROUND, diff: a...no adjoin
title: ALBA -黎明-, diff: n...no adjoin
title: ALBA -黎明-, diff: h...no adjoin
title: ALBA -黎明-, diff: a...no adjoin
title: ALBIDA, diff: n...no adjoin
title: ALBIDA, diff: h...no adjoin
titl

title: Beginning of life(THE GROUND PULSE MIX), diff: h...has adjoin !!!
title: Beginning of life(THE GROUND PULSE MIX), diff: a...has adjoin !!!
title: Beginning of life, diff: n...no adjoin
title: Beginning of life, diff: h...no adjoin
title: Be in my paradise, diff: n...no adjoin
title: Be in my paradise, diff: h...no adjoin
title: being torn the sky, diff: n...no adjoin
title: being torn the sky, diff: h...no adjoin
title: being torn the sky, diff: a...no adjoin
title: believe...?, diff: n...no adjoin
title: believe...?, diff: h...no adjoin
title: Believe In Me, diff: n...no adjoin
title: Believe In Me, diff: h...no adjoin
title: Believe In Me, diff: a...no adjoin
title: B4U(BEMANI FOR YOU MIX), diff: n...no adjoin
title: B4U(BEMANI FOR YOU MIX), diff: h...no adjoin
title: B4U(BEMANI FOR YOU MIX), diff: a...has adjoin !!!
title: B4U(BEMANI FOR YOU MIX), diff: l...no adjoin
title: Be quiet, diff: n...no adjoin
title: Be quiet, diff: h...has adjoin !!!
title: Be quiet, diff: a...no a

...

title: SCORE, diff: h...no adjoin
title: SCORE, diff: a...no adjoin
title: Skyscraper, diff: n...has adjoin !!!
title: Skyscraper, diff: h...no adjoin
title: Scars of FAUNA, diff: n...no adjoin
title: Scars of FAUNA, diff: h...no adjoin
title: Scars of FAUNA, diff: a...no adjoin
title: Scripted Connection⇒ A mix, diff: n...no adjoin
title: Scripted Connection⇒ A mix, diff: h...no adjoin
title: Scripted Connection⇒ A mix, diff: a...has adjoin !!!
title: Scripted Connection⇒ H mix, diff: n...no adjoin
title: Scripted Connection⇒ H mix, diff: h...has adjoin !!!
title: Scripted Connection⇒ H mix, diff: a...no adjoin
title: Scripted Connection⇒ N mix, diff: n...no adjoin
title: Scripted Connection⇒ N mix, diff: h...has adjoin !!!
title: Scripted Connection⇒ N mix, diff: a...has adjoin !!!
title: Scripted Connection⇒, diff: n...no adjoin
title: Scripted Connection⇒, diff: h...has adjoin !!!
title: Scripted Connection⇒, diff: a...has adjoin !!!
title: Secret of Love, diff: n...no adjoin
title

title: So Real, diff: h...has adjoin !!!
title: So Real, diff: a...no adjoin
title: Sorrows, diff: n...no adjoin
title: Sorrows, diff: h...no adjoin
title: Sorrows, diff: a...no adjoin
title: S.O.S., diff: n...no adjoin
title: S.O.S., diff: h...has adjoin !!!
title: S.O.S.(THE TIGER TOOK MY FAMILY), diff: n...has adjoin !!!
title: S.O.S.(THE TIGER TOOK MY FAMILY), diff: h...has adjoin !!!
title: Sounds Of Summer, diff: n...no adjoin
title: Sounds Of Summer, diff: h...no adjoin
title: Sounds Of Summer, diff: a...no adjoin
title: State Of The Art, diff: n...no adjoin
title: State Of The Art, diff: h...has adjoin !!!
title: State Of The Art, diff: a...has adjoin !!!
title: State Of The Art, diff: l...no adjoin
title: Space Time, diff: n...no adjoin
title: Space Time, diff: h...no adjoin
title: Space Time, diff: a...no adjoin
title: SPARK !, diff: n...no adjoin
title: SPARK !, diff: h...no adjoin
title: SPARK !, diff: a...has adjoin !!!
title: SPARK ! -essential RMX-, diff: n...no adjoin
t

title: The Biggest Roaster, diff: h...no adjoin
title: THE SAFARI, diff: n...has adjoin !!!
title: THE SAFARI, diff: h...has adjoin !!!
title: The Sampling Paradise, diff: n...no adjoin
title: The Sampling Paradise, diff: h...no adjoin
title: The Sampling Paradise, diff: a...no adjoin
title: the shadow, diff: n...no adjoin
title: the shadow, diff: h...no adjoin
title: the shadow, diff: a...no adjoin
title: tablets, diff: n...no adjoin
title: tablets, diff: h...has adjoin !!!
title: tablets, diff: a...has adjoin !!!
title: TA・DA ☆ YO・SHI, diff: n...no adjoin
title: TA・DA ☆ YO・SHI, diff: h...no adjoin
title: TA・DA ☆ YO・SHI, diff: a...no adjoin
title: Take It Easy, diff: n...has adjoin !!!
title: Take It Easy, diff: h...has adjoin !!!
title: Take My Life, diff: n...no adjoin
title: Take My Life, diff: h...no adjoin
title: Take My Life, diff: a...no adjoin
title: Take Me Higher, diff: n...no adjoin
title: Take Me Higher, diff: h...no adjoin
title: Take Me Higher, diff: a...no adjoin
title:

title: the trigger of innocence, diff: n...no adjoin
title: the trigger of innocence, diff: h...no adjoin
title: the trigger of innocence, diff: a...no adjoin
title: tripping contact(teranoid&amp;MC Natsack Remix), diff: n...no adjoin
title: tripping contact(teranoid&amp;MC Natsack Remix), diff: h...no adjoin
title: tripping contact(teranoid&amp;MC Natsack Remix), diff: a...no adjoin
title: TRIP MACHINE PhoeniX, diff: n...no adjoin
title: TRIP MACHINE PhoeniX, diff: h...no adjoin
title: TRIP MACHINE PhoeniX, diff: a...no adjoin
title: tripping contact, diff: n...no adjoin
title: tripping contact, diff: h...has adjoin !!!
title: TROOPERS, diff: n...no adjoin
title: TROOPERS, diff: h...no adjoin
title: TROOPERS, diff: a...no adjoin
title: Tropical April, diff: n...no adjoin
title: Tropical April, diff: h...no adjoin
title: Tropical April, diff: a...no adjoin
title: Troposphere, diff: n...no adjoin
title: Troposphere, diff: h...no adjoin
title: Troposphere, diff: a...no adjoin
title: True

title: Winning Eleven9 Theme(IIDX EDITION), diff: n...no adjoin
title: Winning Eleven9 Theme(IIDX EDITION), diff: h...has adjoin !!!
title: WISE UP!, diff: n...no adjoin
title: WISE UP!, diff: h...no adjoin
title: WISE UP!, diff: a...no adjoin
title: wish, diff: n...no adjoin
title: wish, diff: h...has adjoin !!!
title: wish, diff: a...has adjoin !!!
title: WISH(EUROBEAT MIX), diff: n...no adjoin
title: WISH(EUROBEAT MIX), diff: h...has adjoin !!!
title: WISH(EUROBEAT MIX), diff: a...has adjoin !!!
title: with me..., diff: n...no adjoin
title: with me..., diff: h...no adjoin
title: with me..., diff: a...no adjoin
title: with you…, diff: n...no adjoin
title: with you…, diff: h...no adjoin
title: with you…, diff: a...no adjoin
title: Wanna Party?, diff: n...no adjoin
title: Wanna Party?, diff: h...no adjoin
title: Wanna Party?, diff: a...has adjoin !!!
title: Wonder Girl feat. Kanae Asaba, diff: n...no adjoin
title: Wonder Girl feat. Kanae Asaba, diff: h...no adjoin
title: Wonder Girl fe

In [114]:
with open("adjoins.json", "w") as f:
    json.dump(adjoin_scores, f, indent=2, ensure_ascii=False)

In [ ]:
with open("adjoins.json", "r") as f:
    adjoin_scores = json.load(f)

In [115]:
def json2table(adjoin_keys, judge):
    side_header = '<th class=sc_count rowspan="{num_adjoin}", >{side}</th>'
    adjoin_ph = '<td class=sc_count>{key}</td> <td class=sc_count>{times}回</td>'

    if not ("隣接皿" in judge or "無理皿" in judge):
        return "<span class=mucha>無茶皿</span>"
    html_tbl = "<details> <summary>{summary}</summary> <table class=sc_count>".format(summary=judgeColoring(judge))
    for side in adjoin_keys:
        for idx, key in enumerate(adjoin_keys[side]):
            html_tbl += "<tr>"
            if idx == 0:
                num_adjoin = len(adjoin_keys[side].keys())
                html_tbl += side_header.format(num_adjoin=num_adjoin, side=side)
            html_tbl += adjoin_ph.format(key=key, times=adjoin_keys[side][key])
            html_tbl += "</tr>"
    html_tbl += "</table></details>"
    return html_tbl

def char2diff(char_diff):
    index = {"n":"<span class=nml>Normal</span>", 
             "h":"<span class=hpr>Hyper</span>", 
             "a":"<span class=ant>Another</span>", 
             "l":"<span class=leg>Leggendaria</span>"}
    return index[char_diff]

def judgeColoring(judge):
    _ret = ""
    for judge_sp in judge.split(", "):
        for det_char, class_name in zip(["無理皿", "無茶皿", "隣接皿"], ["muri", "mucha", "adjoin"]):
            if det_char in judge_sp:
                _ret += "<span class={class_name}>{judge}</span>".format(class_name=class_name, judge=judge_sp)
    return _ret


In [116]:
#debug
df = pd.DataFrame(adjoin_scores)
df["for_sort"] = df["title"].str.lower()
dd = df.sort_values(["acin", "for_sort"], ascending=[False, True])
# for idx, x in dd.iterrows():
#     print(x)
for x in scores:
    if x[-1]:
        print(x[-1])
print(len(scores))

(1st)
(1st)
(CS DJ TROOPERS)
(CS DJ TROOPERS)
(CS DJ TROOPERS)
(CS3rd〜8th)
(CS3rd〜8th)
(CS3rd〜8th)
(2nd)
(CS HAPPY SKY)
(CS HAPPY SKY)
(CS HAPPY SKY)
(CS10th)
(CS10th)
(CS10th)
(CS)
(CS)
(CS)
(SIRIUS)
(SIRIUS)
(SIRIUS)
(CS DistorteD)
(CS DistorteD)
(CS DistorteD)
(CS7th)
(CS7th)
(CS3rd)
(CS3rd)
(CS3rd)
(1st)
(CS GOLD)
(CS GOLD)
(CS 9th)
(CS 9th)
(CS7th)
(CS7th)
<span style='font-size:8pt'>(SIRIUS)</span>
<span style='font-size:8pt'>(SIRIUS)</span>
<span style='font-size:8pt'>(SIRIUS)</span>
(CS 3rd)
(CS 3rd)
(CS 3rd)
(substream)
(substream)
(TaQ)
(TaQ)
(RYOTA MITSUNAGA)
(RYOTA MITSUNAGA)
(CS DistorteD)
(CS DistorteD)
(CS DistorteD)
(CS3rd)
(CS3rd)
(CS3rd)
(CS6th・8th)
(CS6th・8th)
(AC DJ TROOPERS)
(AC DJ TROOPERS)
(AC DJ TROOPERS)
(AC GOLD)
(AC GOLD)
(AC GOLD)
(CS3rd)
(CS3rd)
(US beatmania)
(US beatmania)
(US beatmania)
(GOLD)
(GOLD)
(GOLD)
(CS3rd)
(CS3rd)
(CS7th)
(CS7th)
(CS7th)
(CS DistorteD)
(CS DistorteD)
(CS DistorteD)
(CS3rd)
(CS3rd)
(CS5th)
(CS5th)
(CS3rd)
(CS3rd)
(US beatmania)
(

In [117]:
header = ["Title", "Difficulty", "Lv", "Type"]
table = \
"""<table><tr class="header title"><th class=header>Title</th> <th class="header diff">Difficulty</th> <th class="header lv">☆</th> <th class="header type">Type</th></tr>"""
df = pd.DataFrame(adjoin_scores)
df["for_sort"] = df["title"].str.lower()
df_sorted = df.sort_values(["acin", "for_sort"], ascending=[False, True])
for idx, info in df_sorted.iterrows():
    title = info["title"] + " " + info["subtitle"] if info["subtitle"] else info["title"]
    data = [title, char2diff(info["diff"]), info["level"], json2table(info["adjoin_keys"], info["judge"])]
    row = "<tr>" if info["acin"] else "<tr class=del>"
    for cell_data in data:
        row += "<td>{cell}</td>".format(cell=cell_data)
    row += "</tr>"
    table += row

table.replace("\'", "&#39;")


'<table><tr class="header title"><th class=header>Title</th> <th class="header diff">Difficulty</th> <th class="header lv">☆</th> <th class="header type">Type</th></tr><tr><td>100% minimoo-G</td><td><span class=ant>Another</span></td><td>11</td><td><details> <summary><span class=adjoin>隣接皿</span></summary> <table class=sc_count><tr><th class=sc_count rowspan="1", >1p</th><td class=sc_count>1</td> <td class=sc_count>1回</td></tr></table></details></td></tr><tr><td>19,November</td><td><span class=nml>Normal</span></td><td>5</td><td><details> <summary><span class=adjoin>隣接皿</span></summary> <table class=sc_count><tr><th class=sc_count rowspan="1", >1p</th><td class=sc_count>1</td> <td class=sc_count>2回</td></tr></table></details></td></tr><tr><td>19,November</td><td><span class=hpr>Hyper</span></td><td>7</td><td><details> <summary><span class=adjoin>隣接皿</span><span class=muri>無理皿</span></summary> <table class=sc_count><tr><th class=sc_count rowspan="3", >1p</th><td class=sc_count>4</td> <t

In [118]:
with open("modules/empty.html", "r", encoding="utf-8") as f:
    htmls = f.read()
head, tail = htmls.split("<!-- pythoh_split -->")
with open("index.html", "w", encoding="utf-8") as f:
    f.write("{head}\n{table}\n{tail}".format(head=head, table=table, tail=tail))

In [95]:
df_sorted.to_csv("test.csv")